In [7]:
import torch.nn as nn
import torch

inputs = torch.Tensor(1,1,28,28)

In [8]:
conv1 = nn.Conv2d(1,32,3,padding=1)
pool = nn.MaxPool2d(2)
conv2 = nn.Conv2d(32,64,3,padding=1)

out = conv1(inputs)
print(out.shape)

torch.Size([1, 32, 28, 28])


In [9]:
out = pool(out)
print(out.shape)

torch.Size([1, 32, 14, 14])


In [10]:
out = conv2(out)
print(out.shape)

torch.Size([1, 64, 14, 14])


In [11]:
out = pool(out)
print(out.shape)

torch.Size([1, 64, 7, 7])


In [12]:
out = out.view(out.size(0),-1)
print(out.shape)

torch.Size([1, 3136])


In [13]:
fc = nn.Linear(3136,10)
out = fc(out)
print(out.shape)

torch.Size([1, 10])


In [26]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

device = 'cpu'

torch.manual_seed(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                            batch_size=batch_size,
                                            shuffle = True,
                                            drop_last=True)
                                        
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc = nn.Linear(7*7*64,10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)

        out = out.view(out.size(0),-1)
        out = self.fc(out)
        return out


model = CNN().to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('Learning start')

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print('[Epoch:{}] cost = {}'.format(epoch+1,avg_cost))
print('Learning Finished')

Learning start
[Epoch:1] cost = 0.22562704980373383
[Epoch:2] cost = 0.06301021575927734
[Epoch:3] cost = 0.046301890164613724
[Epoch:4] cost = 0.03745567053556442
[Epoch:5] cost = 0.03136434406042099
[Epoch:6] cost = 0.025970665737986565
[Epoch:7] cost = 0.021776244044303894
[Epoch:8] cost = 0.018283678218722343
[Epoch:9] cost = 0.016104277223348618
[Epoch:10] cost = 0.013390056788921356
[Epoch:11] cost = 0.009680913761258125
[Epoch:12] cost = 0.009997306391596794
[Epoch:13] cost = 0.008271309547126293
[Epoch:14] cost = 0.0060419063083827496
[Epoch:15] cost = 0.007321633864194155
Learning Finished


In [28]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())

Accuracy: 0.9869999885559082


In [36]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.init

device = 'cpu'

torch.manual_seed(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                            train=False,
                            transform=transforms.ToTensor(),
                            download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                            batch_size=batch_size,
                                            shuffle = True,
                                            drop_last=True)
                                        
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,kernel_size =3 ,stride =1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )

        self.fc1 = nn.Linear(3*3*128,625, bias =True)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(625,10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc1.weight)
        torch.nn.init.xavier_uniform_(self.fc2.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)

        out = out.view(out.size(0),-1)
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        return out


model = CNN().to(device)
print(model)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('Learning start')

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)

        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost+=cost/total_batch
    print('[Epoch:{}] cost = {}'.format(epoch+1,avg_cost))
print('Learning Finished')

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1152, out_features=625, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=625, out_features=10, bias=True)
)
Learning start
[Epoch:1] cost = 0.16050949692726135
[Epoch:2] cost = 0.04232269152998924
[Epoch:3] cost = 0.030048314481973648
[Epoch:4] cost = 0.02208554930984974
[Epoch:5] cost = 0.01883023791015148
[Epoch:6] cost = 0.01428308617323637
[Epoc

In [37]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:',accuracy.item())

C:\Anaconda\lib\site-packages\torchvision\datasets\mnist.py:67: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
Accuracy: 0.9908999800682068
